<a href="https://colab.research.google.com/github/SuriyaPriya17/Intelligent_system/blob/main/PromptEngineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import pipeline
#Zero-shot prompting
classifier=pipeline("zero-shot-classification",model="facebook/bart-large-mnli")
sequence="I love writing code in python and building machine learning models."
labels=["sports","technology","cooking","education"]
result=classifier(sequence, candidate_labels=labels)
print(result)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


{'sequence': 'I love writing code in python and building machine learning models.', 'labels': ['technology', 'education', 'sports', 'cooking'], 'scores': [0.9677100777626038, 0.015067821368575096, 0.010494163259863853, 0.006727938074618578]}


In [2]:
from transformers import pipeline

generator = pipeline("text2text-generation", model="google/flan-t5-base")

prompt = """
Classify the given sentence into one of the following topics:
[sports, technology, cooking, education]

Examples:
Sentence: "He played football with his friends." → sports
Sentence: "She baked a cake for her family." → cooking
Sentence: "Students learned about photosynthesis today." → education

Now classify the following sentence:
"I love eatting parotta."
Answer:
"""

result = generator(prompt, max_length=50)
print("Few-shot Result:")
print(result[0]['generated_text'])


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Few-shot Result:
cooking


In [ ]:
pip install transformers torch sentencepiece

In [3]:
# 1. Install (run once in Colab / local)
# !pip install -q transformers torch sentencepiece

# 2. Imports
from transformers import pipeline
import re
import torch

# 3. Load flan-t5-base (fast on CPU)
print("Loading flan-t5-base …")
generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    device=0 if torch.cuda.is_available() else -1
)

# 4. **Reason-first** prompt (forces step-by-step)
reason_first_prompt = """
You are a classification assistant. ALWAYS follow this exact format:

1. Reasoning: <explain every word of the sentence and why it points to a topic>
2. Final label: <one of: sports, technology, cooking, education> (lowercase, no brackets)

Topics: sports, technology, cooking, education

Example 1:
Sentence: "I scored a goal in soccer yesterday."
1. Reasoning: "scored a goal" and "soccer" are sport-specific terms → sports.
2. Final label: sports

Example 2:
Sentence: "I enjoy baking chocolate chip cookies with my kids."
1. Reasoning: "baking", "cookies", "kids" → preparing food → cooking.
2. Final label: cooking

Example 3:
Sentence: "I'm studying calculus for my university exam."
1. Reasoning: "studying", "calculus", "university exam" → academic learning → education.
2. Final label: education

Now classify:
Sentence: "I love writing code in python and building machine learning models."

Respond **exactly** in the numbered format above.
"""

# 5. Generate with *light* sampling
print("\nGenerating …")
out = generator(
    reason_first_prompt,
    max_new_tokens=220,
    do_sample=True,          # needed for reasoning
    temperature=0.6,         # low → stays on-track but explores
    top_p=0.93,
    repetition_penalty=1.2
)

raw = out[0]["generated_text"].strip()

print("\n" + "="*70)
print("RAW MODEL OUTPUT")
print("="*70)
print(raw)
print("="*70 + "\n")

# 6. Extract label (very robust)
m = re.search(
    r"(?:Final label|2\.)\s*:\s*([A-Za-z]+)|"
    r"\b(sports|technology|cooking|education)\b(?=\s*$)",
    raw,
    re.IGNORECASE
)
label = m.group(1).lower() if m and m.group(1) else (m.group(2).lower() if m and m.group(2) else "not found")

print(f"Parsed final label: \033[92m{label}\033[0m")

Loading flan-t5-base …


Device set to use cpu



Generating …

RAW MODEL OUTPUT
1. Reasoning: I love python and building machine learning models. 2. Final label: education

Parsed final label: education
